In [3]:
import numpy as np
np.logspace(-2, 5, 8) 

array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05])

In [1]:
from Fairness_aware_model import FairnessAwareModel
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split

In [8]:
df = pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6], "C":[1,2,3]})

In [25]:
new_colnames

['which_set', 0.1, 10000000000.0]

In [17]:
df.columns = new_colnames

TypeError: Index(...) must be called with a collection of some kind, None was passed

In [6]:
df

,bonjour,hi
0,1,4
1,2,5
2,3,6


In [ ]:
dataCar = pd.read_csv('dataCar_clean.csv') # contient le train-test split pour comparer avec R au besoin
#clm = dataCar['clm']
#Xtrain, test, _, _ = train_test_split(dataCar, clm, test_size=0.2)
train=dataCar.loc[dataCar['train']==1]
test=dataCar.loc[dataCar['train']==0]

# Modèle baseline avec la méthode bfgs

In [ ]:
mod = smf.glm(formula="clm ~ veh_value + veh_body + C(veh_age) + area + gender + C(agecat)", family=sm.families.Binomial(), data=train)
res = mod.fit(method="bfgs")
print(res.summary()) # donne la même chose qu'en R

In [ ]:
protected_attributes = ["gender", "agecat"]
# Standard scaling for regression
binary_answer = dataCar["clm"].values
numclaim = dataCar["numclaims"].values
reg_claim = dataCar["claimcst0"].values

In [ ]:
def hot_encoder(data, optional_columns):
    categorical_cols = []
    for column in data.columns:
            if data[column].dtype == object or column in optional_columns:
                categorical_cols.append(column)

    to_return = pd.get_dummies(data, columns = categorical_cols)
    return to_return

dataCar_encoded = hot_encoder(dataCar, ["gender", "agecat"])

dataCar_encoded = dataCar_encoded.drop(["clm", "numclaims", "claimcst0", "veh_body_BUS","area_A","exposure", "gender_F", "agecat_1"], axis=1)

In [ ]:
train_encoded = dataCar_encoded.loc[dataCar_encoded['train']==1]
test_encoded = dataCar_encoded.loc[dataCar_encoded['train']==0]
y = dataCar['clm']

y_train = dataCar.loc[dataCar['train']==1, 'clm']
y_test = dataCar.loc[dataCar['train']==0, 'clm']
X_train_encoded = train_encoded.drop(["train"], axis=1)
X_test_encoded = test_encoded.drop(["train"], axis=1)

In [ ]:
protected_attributes = [np.where(dataCar.columns.values == i)[0][0] for i in protected_attributes] # position des colonnes d'attributs protégés dans les données

In [ ]:
fam = FairnessAwareModel(regularization=0, protected_attributes=protected_attributes, family="binomial", offset=1)
fam.fit(np.column_stack([np.ones(X_train_encoded.shape[0]), np.array(X_train_encoded)]), y_train.values) # il faut inclute un "intercept", donc colonne de 1

# Comparaison avec `statsmodel`

In [ ]:
model2 = sm.Logit(y_train, sm.add_constant(X_train_encoded)).fit()

In [ ]:
print(model2.summary())

In [ ]:
pd.DataFrame({"variable":["intercept"] + X_train_encoded.columns.tolist(), "beta":fam.beta, "beta_statsnodel":model2.params})